In [1]:
'''
RAG - Decomposition question to sub-questions and aggregating each sub-question/answar pair
as part of prompt context in the next sub-question prompt till final.

RAG - Decomposition question to sub-questions, and provide the final answer from the 
retrieved documents of each sub-question.

Decomposition translation is different from multi-query or fusion, instead of 
creating similar questions, it creates sub-questions from the original question. 

step_to_step_decomposition, at end, is based on the LangChain Rag youtube tutorial below, 
the code from the tutorial is listed in the first half of the ipython notebook. 

This function follows the same logic from the tutorial but without the abstract of 
Chain/Pipeline. This approach helps me to understand the sub-questions creation, 
llm processes and also can improve the final prompt preparation. 

Of course, the langsmith is a great tool as well. 

I may be wrong, but it seems to me the separate the sub-questions as individual questions
can return more accurate results than the aggregation approach. 
However, the aggregation decomposition approach provide more details in answer.


Input arguments to the step_to_step_fusion are
1. Quesition: question string
2. Additional_background:  the helping instructions you want the ChatPGT to return the Q&A 
with a better answer.
3. Aggregation: 1 means aggregating sub-questions as prompt for the next sub-question Q&A


Please review the RAG tutorial from Langchain in details (part 7 
- decompistion query translation)
https://www.youtube.com/watch?v=h0OPWlEOank

'''
#RAG-- Common Rag
#Query Translation:   Multi-Query, Fusion and Decompsoition 
import os
from dotenv import load_dotenv
load_dotenv()

#if (GCP_PROJECT_ID == None): print ("Not set")
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'false'  #true for trace
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ["LANGCHAIN_PROJECT"] = "RAG - Decomposition-Query-Translation"

In [2]:
#RAG-- Common Rag
#Query Translation:   Multi-Query, Fusion and Decompsoition 
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
text_splitter = CharacterTextSplitter()
question = "Please list all the documents that contains the robotic experience"
question="List all candidates by name that who have Java work experience?"

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
#RAG-- Common Rag
#Query Translation:   Multi-Query, Fusion and Decompsoition 
#Loading PDF resumes from local instead of the Web

import uuid
import hashlib
from PyPDF2 import PdfReader
def extract_pdf_text(file_path):
    pdf_file = PdfReader(file_path)
    text_data = ''
    for pg in pdf_file.pages:
        text_data += pg.extract_text()
    return text_data


resume_dir = ".\\docs\\"
pdf_text = []


#Change 1
embeddings = OpenAIEmbeddings()
vectorstore = Chroma("langchain", embeddings)



def create_uuid_from_string(val: str):
    hex_string = hashlib.md5(val.encode("UTF-8")).hexdigest()
    return uuid.UUID(hex=hex_string)


for file in os.listdir(resume_dir):
    filepath = os.path.join(resume_dir,file)
    collection = vectorstore.get('langchain')
    if (filepath.endswith('.pdf')): 
        #Change 2
        pdf_text=[]
        pdf_text.append ("File Name: "+filepath+"  \n"+extract_pdf_text(filepath))
        split_docs = text_splitter.create_documents(pdf_text)
        existing = vectorstore.get(file)
       
        if (existing['ids'] != [] and existing['ids'][0] == file):
            print("Deleting Duplication .....", file)
            vectorstore.delete(file)
        # Add documents back to collection
        try:
            #print("Split Doc   ", split_docs)
            #Need to provide IDS list to add_documents, otherwise, it will only pick up the first character of the file name
            langchain_ids = vectorstore.add_documents(ids=[file], documents=split_docs) 
            print("Adding Langchain ID - ", langchain_ids, " File Name - ", file)
            #langchain_ids should be equal to file str
        except:
            #print("Again....Deleting Duplication .....", file)
            vectorstore.delete(file)
            #print("Existing... ", existing['ids'])
            print("Can't add.. ", file)




In [ ]:
#Retrieving
#RAG-- Common Rag

#vectorstore = Chroma.from_documents(documents=split_docs, embedding = embedding)
#retriever=vectorstore.as_retriever(search_kwargs={"k": 2}, max_tokens_limit=10000)
retriever=vectorstore.as_retriever(search_kwargs={"k": 2}, max_tokens_limit=10000,
                                   collection_metadata={"hnsw:M": 1024,"hnsw:ef": 64})

In [ ]:
#Decomposition - Method 1, Aggreating subquestions.
from langchain.prompts import ChatPromptTemplate
#question = "Please list all the documents that contains the robotic experience"
#Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question about a resume. \n
The goal is to break down the input question into a set of sub-question about the experience listed in a resume, and each sub-question that can be answers in separately \n
However, you must keep the principle of the origin question in sub-question \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [ ]:
#Decomposition - Method 1  Continue
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
#question="How long has each candidate been involved with robotics, and what roles have they held in this field?"
#question ="Please list all the candidate name that who has the robotic experience"
# LLM
question="List all candidates by name that who have Java work experience?"
llm = ChatOpenAI(temperature=0.2)
# ***relaxing temperature between 0 - 0.5 may have a better sub-questions generated.
# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))
# Run
questions = generate_queries_decomposition.invoke({"question":question})

In [ ]:
#Decomposition - Method 1 Continue

context = """Each document (or resume) contains

0. The file name is listed in the first line,
1. The resume contains the name, the phone number and the address of a person, \n
2. The resume contains the degree and attending university and years of graduation.  \n
3. The resume contains the work history of all the companies a person worked for.  
Each job may contain the start and end dates, and job title. If the end date is 
missing then, the job is the current job.  
4. Each job has the job titles, and associated responsibilities or experience of 
that person. Each job is independent from other jobs in the same document. 
5. The resume may contain any awards or social networking information of this person. \n
6. The first job should have the farest year from current date, the last or current job 
has the closest to the current date.  From the last and first job you should be able 
to calculate the total year a person has worked for each job and total years of a person
has been worked.\n

"""

template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the answer of the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [ ]:
#Decomposition - Method 1 Continue
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    #print("format String...", formatted_string.strip(), "\n")
    return formatted_string.strip()

# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

q_a_pairs = ""

for q in questions:
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair
print("Aggregation answer is ...", answer)

In [ ]:
#Decomposition - Method 2, treat sub questions as separated question and 
#Answer each sub-question individually before passing final RAG Q&A

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG prompt
#prompt_rag = hub.pull("rlm/rag-prompt")
#retriever.get_relevant_documents(sub_question)
#calling invoke direclty which will call get_relevant_documents

'''
#Original Codes
def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        print("subquestion::: ", sub_question)
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        #Same as Invoke, since Invoke will call the get_relevant_documents evatually 
        #Use retrieved documents and sub-question separately in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        #print ("answer:::", answer)
        rag_results.append(answer)
    
    return rag_results,sub_questions
'''
question="How long has each candidate been involved with robotics, and what roles have they held in this field?"
template = """You are a helpful assistant that generates multiple sub-questions related to an input question about a resume. \n
The goal is to break down the input question into a set of sub-question about the experience listed in a resume, and each sub-question that can be answers in separately \n
However, you must keep the principle of the origin question in sub-question \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_rag2 = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(temperature=0.2)
# ***relaxing temperature between 0 - 0.5 may have a better sub-questions generated.
# Chain
generate_queries_decomposition2 = ( prompt_rag2 | llm | StrOutputParser() | (lambda x: x.split("\n")))
questions = generate_queries_decomposition2.invoke({"question":question})

context = """Each document (or resume) contains

0. The file name is listed in the first line,
1. The resume contains the name, the phone number and the address of a person, \n
2. The resume contains the degree and attending university and years of graduation.  \n
3. The resume contains the work history of all the companies a person worked for.  
Each job may contain the start and end dates, and job title. If the end date is 
missing then, the job is the current job.  
4. Each job has the job titles, and associated responsibilities or experience of 
that person. Each job is independent from other jobs in the same document. 
5. The resume may contain any awards or social networking information of this person. \n
6. The first job should have the farest year from current date, the last or current job 
has the closest to the current date.  From the last and first job you should be able 
to calculate the total year a person has worked for each job and total years of a person
has been worked.\n

"""

template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is additional context relevant to the answer of the question: 

\n --- \n {context} \n --- \n
Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

#decomposition_prompts by removing the last question and answer pair in the prompt by comparing to the method 1
'''Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n
'''
decomposition_prompt2 = ChatPromptTemplate.from_template(template)
answers = []
for q in questions:
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question") } 
    | decomposition_prompt2
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q})
    #q_a_pair = format_qa_pair(q,answer)
    #q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair
    #print("answer...", answer)
    answers.append(answer)

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
# with 2nd method, the subquestions generation is the same as the 1st method
#answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [ ]:
#Decomposition - Method 2 Continue
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

template = """Here is a set of quest pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

'''
also, can be included following as additional_background 
the context contains
0. The file name is listed in the first line,
1. The resume contains the name, the phone number and the address of a person, \n
2. The resume contains the degree and attending university and years of graduation.  \n
3. The resume contains the work history of all the companies a person worked for.  
Each job may contain the start and end dates, and job title. If the end date is 
missing then, the job is the current job.  
4. Each job has the job titles, and associated responsibilities or experience of 
that person. Each job is independent from other jobs in the same document. 
5. The resume may contain any awards or social networking information of this person. \n
6. The first job should have the farest year from current date, the last or current job 
has the closest to the current date.  From the last and first job you should be able 
to calculate the total year a person has worked for each job and total years of a person
has been worked.\n : {question}
'''

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag = final_rag_chain.invoke({"context":context,"question":question})
print ("final_separation_rag is ",  final_rag)

In [ ]:
# Query Translation:  step_by_step_decomposition with aggregation and separation as one function

import os
from dotenv import load_dotenv
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import uuid
import hashlib
from PyPDF2 import PdfReader
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
#question = "Please list all the documents that contains the robotic experience"
#question="How long has each candidate been involved with robotics, and what roles have they held in this field?"

def extract_pdf_text(file_path):
    pdf_file = PdfReader(file_path)
    text_data = ''
    for pg in pdf_file.pages:
        text_data += pg.extract_text()
    return text_data

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    #print("format string... ", formatted_string, "\n")
    return formatted_string.strip()
    
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

def step_by_step_decomposition(question, additional_background, selection):
    load_dotenv()

#if (GCP_PROJECT_ID == None): print ("Not set")
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
    os.environ['LANGCHAIN_TRACING_V2'] = 'false'  #true for trace
    os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
    os.environ["LANGCHAIN_PROJECT"] = "RAG - Decomposition-Query-Translation"
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_splitter = CharacterTextSplitter()
    resume_dir = ".\\docs\\"
    pdf_text = []
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma("langchain", embeddings)
    for file in os.listdir(resume_dir):
        filepath = os.path.join(resume_dir,file)
        collection = vectorstore.get('langchain')
        if (filepath.endswith('.pdf')): 
        #Change 2
            pdf_text=[]
            pdf_text.append ("File Name: "+filepath+"  \n"+extract_pdf_text(filepath))
            split_docs = text_splitter.create_documents(pdf_text)
            existing = vectorstore.get(file)
            #print("adding... ", filepath)

            if (existing['ids'] != [] and existing['ids'][0] == file):
                #print("Deleting Duplication .....", file)
                vectorstore.delete(file)
            # Add documents back to collection
            try:
                #print("Split Doc   ", split_docs)
                #Need to provide IDS list to add_documents, otherwise, it will only pick up the first character of the file name
                langchain_ids = vectorstore.add_documents(ids=[file], documents=split_docs) 
                #print("Adding Langchain ID - ", langchain_ids, " File Name - ", file)
                #langchain_ids should be equal to file str
            except:
                #print("Again....Deleting Duplication .....", file)
                vectorstore.delete(file)
                #print("Existing... ", existing['ids'])
                print("Can't add.. ", file)
                
    retriever=vectorstore.as_retriever(max_tokens_limit=20000,
                                       collection_metadata={"hnsw:M": 1024,"hnsw:ef": 64})
# Adding search_kwargs or max_tok to avoid the following runtime error
# Cannot return the results in a contigious 2D array. Probably ef or M is too small"
# search_kwargs={"k": 2}, 
    template_subquestions = """You are a helpful assistant that generates multiple sub-questions related to an input question about a resume. \n
The goal is to break down the input question into a set of sub-question about the experience listed in a resume, and each sub-question that can be answers in separately \n
However, you must keep the principle of the origin question in sub-question \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""

    prompt_decomposition = ChatPromptTemplate.from_template(template_subquestions)

    #llm = ChatOpenAI(temperature=0.2)
    # ***relaxing temperature between 0 - 0.5 may have a better sub-questions generated.

    # Chain
    #generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

    
    prompt_decomposition_output =  prompt_decomposition.invoke(question)
    #print("Output after prompt_perspectives:", prompt_decomposition_output)

    chat_openai_output = ChatOpenAI(temperature=0.2)(prompt_decomposition_output)
        ##print("Output after ChatOpenAI:", chat_openai_output)
    
    parser_output = StrOutputParser().invoke(chat_openai_output)
        #print("output after StrOutputParser:", parser_output)
        #str_output_parser_output = output_parser(chat_openai_output)
        #print("Output after StrOutputParser:", str_output_parser_output)
    questions = parser_output.split("\n")
    #print("questions len...", len(questions), "...", questions)
    

    template = """Here is the question you need to answer:

    \n --- \n {question} \n --- \n

    Here is any available background question + answer pairs:

    \n --- \n {q_a_pairs} \n --- \n

    Here is additional background \n --- \n {background}
    and context relevant to the answer of the question: 

    \n --- \n {context} \n --- \n
    Use the above context and any background question + answer pairs to answer the question: \n {question}
    """

    decomposition_prompt = ChatPromptTemplate.from_template(template)
    
    #Decomposition - Method 1 Continue




# llm
    llm = ChatOpenAI(temperature=0.2)
    answers = []
    q_a_pairs = ""
    for q in questions:
        #print("question... ", q, "\n")
        docs = retriever.invoke(q)
        #print("docs... ", docs)
        if (selection == 1):
            aggregations = decomposition_prompt.invoke({"question": q,
                                                        "q_a_pairs": q_a_pairs, 
                                                        "context": docs,
                                                        "background": additional_background})
        else:
            q_a_pairs = ""
            aggregations = decomposition_prompt.invoke({"question": q,
                                                        "q_a_pairs": q_a_pairs,
                                                        "context": docs,
                                                        "background": additional_background})
            
        chat_openai_output = llm(aggregations)
        parser_output = StrOutputParser().invoke(chat_openai_output)
        #print("Parser Output ", parser_output)
        if (selection == 1): 
            final_rag1 = parser_output
            q_a_pair = format_qa_pair(q, parser_output)
            q_a_pairs = q_a_pairs + "\n" + q_a_pair
        else:        
            answers.append(parser_output)
            
        
    if (selection != 1):
        context = format_qa_pairs(questions, answers)
        #print("context....", context)
        template = """Here is a set of quest pairs:

        {context}

        Use these to synthesize an answer to the question: {question}
        """
        prompt = ChatPromptTemplate.from_template(template)
        separation = prompt.invoke({"question": question, "context": context})
        chat_openai_output = llm(separation)
        final_rag2 = StrOutputParser().invoke(chat_openai_output)
        print("\n\n\nFinal Rag - Separation answer is:    ", final_rag2)
    else:
        print("\n\n\nFinal Rag - Aggregation answer is:    ", final_rag1)


In [ ]:
additional_background = """Each document (or resume) may include
0. The file name is listed in the first line,
1. The resume contains the name, the phone number and the address of a person, \n
2. The resume contains the degree and attending university and years of graduation.  \n
3. The resume contains the work history of all the companies a person worked for.  
Each job may contain the start and end dates, and job title. If the end date is 
missing then, the job is the current job.  
4. Each job has the job titles, and associated responsibilities or experience of 
that person. Each job is independent from other jobs in the same document. 
5. The resume may contain any awards or social networking information of this person. \n
6. The first job should have the farest year from current date, the last or current job 
has the closest to the current date.  From the last and first job you should be able 
to calculate the total year a person has worked for each job and total years of a person
has been worked.\n
    
    
"""
additional_background = ""
question="Who have the Java experience?"
step_by_step_decomposition(question, additional_background, 1)

In [ ]:
'''
Final Rag - Aggregation answer is:     Based on the provided background information, the candidate Teresa Technical has worked on Java projects during her Software Development Internship at Pros in Denver, CO. She designed a REST API call in Java, used a topological sorting algorithm on a DAG, and refactored existing API calls to use XML Serializable objects and new API backend methods. Additionally, she has listed proficiency in Java as one of her technical skills. Therefore, Teresa Technical can provide examples of Java code she has written or contributed to based on her experience at Pros.


Final Rag - Separation answer is:     Based on the provided information, both Teresa Technical and Sam C. Yang have significant experience with Java programming. Teresa Technical has completed various projects involving Java, such as designing REST API calls, implementing algorithms for data dependency resolution, and refactoring existing API calls. On the other hand, Sam C. Yang has over 20 years of experience in software development and program management, which includes experience in Java programming. Therefore, both individuals have demonstrated their expertise and proficiency in Java through their professional experiences and accomplishments.
'''

In [ ]:
question="Please provide summary of all candidates"
step_by_step_decomposition(question, context, 0)

In [ ]:
'''
Final Rag - Aggregation answer is:     Candidate 1 (Sam C. Yang) has a Master of Science in Computer Engineering from the University of Southern California and a Bachelor of Science in Physics from F. J. U in Taiwan. This educational background provides him with a strong foundation in both computer engineering and physics, which are relevant to the technical aspects of the position. Additionally, his extensive experience in software development and program management aligns well with the requirements of the role.

Candidate 2 (Teresa Technical) holds a Bachelor of Science in Computer Science from The University of Texas at Austin, with relevant coursework in Object-Oriented Programming, Artificial Intelligence, Algorithms, Data Structures, and Data Mining. This educational background equips her with the necessary technical skills and knowledge to excel in the position. Her experience as a Software Development Intern and Online Marketing Intern further demonstrates her practical application of these skills.

Candidate 3 (Daisuke Data Science) has a Bachelor of Science in Biology with a focus on Computational Biology from The University of Texas at Austin. This educational background provides him with a strong foundation in biology and computational science, which are relevant to the data science aspects of the position. His experience as a Data Science Intern at Merck & Co. and as a Data Analyst Intern at Austin Sports Medicine, Orthopedics showcases his practical experience in utilizing data analytics tools and techniques.

Overall, the educational backgrounds of all three candidates contribute to their suitability for the position by providing them with the necessary knowledge, skills, and experience to excel in their respective roles.

Final Rag - Separation answer is:     Summary of all candidates:

Candidate 1 (Sam C. Yang) brings over 20 years of software development and program management experience in various industries, with a strong focus on SaaS/Cloud, Robotics/Sensors/Control, and E-Commerce. His extensive experience in managing cross-functional teams and delivering products from RFP/RFQ to Mass Production makes him a valuable asset for the position.

Candidate 2 (Teresa Technical) holds a Bachelor of Science in Computer Science and has practical experience as a Software Development Intern and Online Marketing Intern. Her proficiency in Java and familiarity with other programming languages, coupled with her research in computational science and engineering, showcases her technical skills and ability to apply them in real-world projects.

Candidate 3 (Daisuke Data Science) has a background in Biology with a focus on Computational Biology, providing him with a strong foundation in data analysis and statistical modeling. His internship experiences in data science and healthcare analytics, along with his research projects in bioinformatics, demonstrate his ability to leverage data for insights and decision-making.

Overall, each candidate brings a unique set of qualifications, skills, and experiences that can contribute to the company's success in different ways. Sam C. Yang's leadership and program management expertise, Teresa Technical's technical proficiency and research background, and Daisuke Data Science's data analysis skills and research experience make them all strong contenders for the position
'''

In [ ]:
'''
additional_background = """Each document (or resume) may include

    0. The file name is listed in the first line,
1. The resume contains the name, the phone number and the address of a person, \n
2. The resume contains the degree and attending university and years of graduation.  \n
3. The resume contains the work history of all the companies a person worked for.  
Each job may contain the start and end dates, and job title. If the end date is 
missing then, the job is the current job.  
4. Each job has the job titles, and associated responsibilities or experience of 
that person. Each job is independent from other jobs in the same document. 
5. The resume may contain any awards or social networking information of this person. \n
6. The first job should have the farest year from current date, the last or current job 
has the closest to the current date.  From the last and first job you should be able 
to calculate the total year a person has worked for each job and total years of a person
has been worked.\n
    
    
"""
'''

additional_background = ""
question="How long has each candidate been involved with robotics, and what roles have they held in this field?"
step_by_step_decomposition(question, additional_background, 0)

In [ ]:
'''
Final Rag - Aggregation answer is:     Candidate 1, Sam C. Yang, has had the following roles in the field of robotics:

1. Miso Robotics - Tech. Program Manager from 11/2021 to 09/2022 (duration: 10 months)
2. Quasar Science - NPI Program Manager from 6/2020 to 11/2021 (duration: 1 year, 5 months)

Candidate 2, Teresa Technical, does not have any specific experience mentioned in the field of robotics.

Candidate 3, Daisuke Data Science, does not have any specific experience mentioned in the field of robotics.

Final Rag - Aggregation answer is:     Candidate 1, Sam C. Yang, has had the following roles in the field of robotics:

1. Miso Robotics - Tech. Program Manager from 11/2021 to 09/2022 (duration: 10 months)
2. Quasar Science - NPI Program Manager from 6/2020 to 11/2021 (duration: 1 year, 5 months)

Candidate 2, Teresa Technical, does not have any specific experience mentioned in the field of robotics.

Candidate 3, Daisuke Data Science, does not have any specific experience mentioned in the field of robotics.
'''

In [ ]:
#cleanup the vectordb
def clean_vectorstore():
    embeddings = OpenAIEmbeddings()
    ectorstore = Chroma("langchain", embeddings)
    ids = vectorstore.get().get('ids')
    len(ids)
    print("ids ", ids)
    for id in ids: 
        docs = vectorstore.get(id)
        print("docs ", docs)
        print(id)
        vectorstore.delete(id)

In [ ]:
clean_vectorstore()